Source api: [OPEN-METEO](https://open-meteo.com/)

In [1]:
import requests as req
import pandas as pd
import matplotlib.pyplot as plt
import requests_cache
from retry_requests import retry
import openmeteo_requests

In [2]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [3]:
weather_folder_path = './data/weather'

In [4]:
years = [('2017-01-01', '2017-12-31'),
         ('2018-01-01', '2018-12-31'),
         ('2019-01-01', '2019-12-31'),
         ('2020-01-01', '2020-12-31'),
         ('2021-01-01', '2021-12-31'),
         ('2022-01-01', '2022-12-31'),
         ('2023-01-01', '2023-12-31'),
         ('2024-01-01', '2024-03-24')]

In [5]:
features = ['temperature_2m',
           'relative_humidity_2m',
           'pressure_msl',
           'surface_pressure',
           'precipitation',
           'rain',
           'cloud_cover',
           'cloud_cover_low',
           'cloud_cover_mid',
           'cloud_cover_high',
           'shortwave_radiation',
           'direct_radiation',
           'diffuse_radiation',
           'sunshine_duration',
           'wind_speed_10m',
           'wind_speed_100m',
           'wind_direction_10m',
           'wind_direction_100m',
           'wind_gusts_10m',]

In [6]:
bp = {'lat': 47.4925, 'long': 19.051389, 'name' : 'BP'}
konst = {'lat': 44.1811, 'long': 28.5476, 'name' : 'Konstanca'}

In [7]:
url = "https://archive-api.open-meteo.com/v1/archive"

In [8]:
def get_data(location, start_date, end_date, features):
    features_str = ','.join(features)
    features_str

    params = {
        "latitude": location.get('lat'),
        "longitude": location.get('long'),
        "start_date": start_date,
        "end_date": end_date,
        "hourly":  features_str
    }
    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]
    hourly = response.Hourly()

    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
        )}
    
    for f in features:
        hourly_data[f"{f}_{location.get('name')}"] = hourly.Variables(features.index(f)).ValuesAsNumpy()

    df = pd.DataFrame(hourly_data)
    return df


In [9]:
locations = [bp, konst]

In [10]:
for loc in locations:
    df_loc = pd.DataFrame()
    for year in years:
        df = get_data(loc, year[0], year[1], features)
        df_loc = pd.concat([df_loc, df])
    if loc == locations[0]:
        df_final = df_loc
    else:
        df_final = pd.merge(df_final, df_loc, on = 'date', how = 'outer')

In [11]:
# rename date to Datetime
df_final = df_final.rename(columns = {'date': 'Datetime'})

In [12]:
df_final

,Datetime,temperature_2m_BP,relative_humidity_2m_BP,pressure_msl_BP,surface_pressure_BP,precipitation_BP,rain_BP,cloud_cover_BP,cloud_cover_low_BP,cloud_cover_mid_BP,...,cloud_cover_high_Konstanca,shortwave_radiation_Konstanca,direct_radiation_Konstanca,diffuse_radiation_Konstanca,sunshine_duration_Konstanca,wind_speed_10m_Konstanca,wind_speed_100m_Konstanca,wind_direction_10m_Konstanca,wind_direction_100m_Konstanca,wind_gusts_10m_Konstanca
0,2017-01-01 00:00:00+00:00,-3.579,95.965530,1029.599976,1014.724121,0.0,0.0,100.000000,96.0,0.0,...,0.0,0.0,0.0,0.0,0.0,16.055353,31.154043,289.653900,292.416199,27.719999
1,2017-01-01 01:00:00+00:00,-4.429,95.937294,1029.400024,1014.480225,0.0,0.0,65.400002,44.0,0.0,...,0.0,0.0,0.0,0.0,0.0,17.317459,31.883888,290.695526,295.407684,29.519999
2,2017-01-01 02:00:00+00:00,-6.079,95.514832,1028.699951,1013.699158,0.0,0.0,61.799999,41.0,0.0,...,0.0,0.0,0.0,0.0,0.0,17.992796,32.862694,289.885254,294.599762,30.960001
3,2017-01-01 03:00:00+00:00,-5.729,95.893539,1028.300049,1013.324524,0.0,0.0,54.599998,32.0,0.0,...,0.0,0.0,0.0,0.0,0.0,18.899143,33.759480,287.744751,291.914917,32.399998
4,2017-01-01 04:00:00+00:00,-5.279,99.244804,1027.900024,1012.955200,0.0,0.0,44.099998,47.0,0.0,...,0.0,0.0,0.0,0.0,0.0,19.211996,33.931789,282.994629,287.281586,32.760002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63355,2024-03-24 19:00:00+00:00,5.421,80.713318,1007.200012,993.114014,0.0,0.0,0.000000,0.0,0.0,...,72.0,0.0,0.0,0.0,0.0,11.304229,26.987997,217.234909,223.919159,20.519999
63356,2024-03-24 20:00:00+00:00,4.171,88.378944,1007.099976,992.952393,0.0,0.0,1.800000,2.0,0.0,...,100.0,0.0,0.0,0.0,0.0,10.799999,26.220753,216.869980,224.443817,18.359999
63357,2024-03-24 21:00:00+00:00,3.721,90.892494,1007.000000,992.831055,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,12.313894,28.284639,217.875046,222.420807,20.160000
63358,2024-03-24 22:00:00+00:00,3.471,90.874847,1006.799988,992.621033,0.0,0.0,11.400001,0.0,11.0,...,1.0,0.0,0.0,0.0,0.0,12.496719,28.817993,258.366394,257.005371,20.519999


In [13]:
df_final.to_csv(f'{weather_folder_path}/weather_{[loc.get("name") for loc in locations]}_{years[0][0],years[-1][-1]}.csv', index = False)